In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.Builder() \
    .appName("data_processor") \
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:8.6.2") \
    .master("local[4]").getOrCreate()

# df = spark.read.format('csv').option('header', 'true').load(
#     'data/complaints_init.csv'
# )

df = spark.read.csv(
    'data/training_init.csv',
    header=True,
    inferSchema=True    
)







In [26]:
INDEX_NAME = 'test-kakfa-ingestion-flow'
df = spark.read.format("org.elasticsearch.spark.sql").option("es.resource", '%s/%s' % (INDEX_NAME, "_doc")) \

df.load().show()

23/11/27 22:18:11 WARN RestClient: Could not verify server is Elasticsearch! ES-Hadoop will require server validation when connecting to an Elasticsearch cluster if that Elasticsearch cluster is v7.14 and up.
23/11/27 22:18:11 WARN Resource: Detected type name in resource [test-kakfa-ingestion-flow/_doc]. Type names are deprecated and will be removed in a later release.
23/11/27 22:18:11 WARN RestClient: Could not verify server is Elasticsearch! ES-Hadoop will require server validation when connecting to an Elasticsearch cluster if that Elasticsearch cluster is v7.14 and up.
23/11/27 22:18:11 WARN Resource: Detected type name in resource [test-kakfa-ingestion-flow/_doc]. Type names are deprecated and will be removed in a later release.
23/11/27 22:18:11 WARN Resource: Detected type name in resource [test-kakfa-ingestion-flow/_doc]. Type names are deprecated and will be removed in a later release.
23/11/27 22:18:11 WARN Resource: Detected type name in resource [test-kakfa-ingestion-flow

In [2]:
class LabelSimulator:
    def __init__(self, label_data_path):
        # self.label_df = pd.read_csv(label_data_path)
        self.label_df = spark.read.csv(
            label_data_path,
            header=True,
            inferSchema=True
        )

    def annotate(self, ids):
        return self.label_df.where(self.label_df['id'].isin(ids)).target
        # return self.label_df[self.label_df.isin(ids)].target

In [23]:
lb = spark.read.csv('data/label.csv', header=True, inferSchema=True)
df = spark.read.csv('data/training_init.csv', header=True, inferSchema=True).drop("target")


In [36]:
import pandas as pd 

lb.join(df, on=['id', 'user']).dropDuplicates(['id', 'user']).show()

+----------+---------------+------+--------------------+--------+--------------------+
|        id|           user|target|                date|    flag|                text|
+----------+---------------+------+--------------------+--------+--------------------+
|1467810369|_TheSpecialOne_|     0|Mon Apr 06 22:19:...|NO_QUERY|@switchfoot http:...|
|1467810672|  scotthamilton|     0|Mon Apr 06 22:19:...|NO_QUERY|is upset that he ...|
|1467810917|       mattycus|     0|Mon Apr 06 22:19:...|NO_QUERY|@Kenichan I dived...|
|1467811184|        ElleCTF|     0|Mon Apr 06 22:19:...|NO_QUERY|my whole body fee...|
|1467811193|         Karoli|     0|Mon Apr 06 22:19:...|NO_QUERY|@nationwideclass ...|
|1467811372|       joy_wolf|     0|Mon Apr 06 22:20:...|NO_QUERY|@Kwesidei not the...|
|1467811592|        mybirch|     0|Mon Apr 06 22:20:...|NO_QUERY|         Need a hug |
|1467811594|           coZZ|     0|Mon Apr 06 22:20:...|NO_QUERY|@LOLTrish hey  lo...|
|1467811795|2Hood4Hollywood|     0|Mon Apr 

In [30]:
ldf_group[ldf_group>1]

id
1467863684    2
1467880442    2
1468053611    2
1468100580    2
1468115720    2
             ..
2193278017    2
2193403830    2
2193428118    2
2193451289    2
2193576442    2
Length: 1685, dtype: int64

In [25]:
# df.where(df['id'].isin(df.id)).show()


df.join(lb, on=['id', 'user']).count()

# lb.where(lb.id.isin(df.id)).count()

10036

In [ ]:
df = (spark.read
      .format( "org.elasticsearch.spark.sql" )
      .option( "es.nodes",   "localhost" )
      .option( "es.port",    5601     )
    #   .option( "es.net.ssl", ssl      )
      .option( "es.nodes.wan.only", "true" )
      .load( f"spark_index/doc" )
     )

In [ ]:
# init model
from typing import Optional
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import HashingTF, IDF, StopWordsRemover, Tokenizer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark import keyword_only
from pyspark.sql.types import StringType, ArrayType, FloatType

import pyspark.sql.functions as F

import re
import string

from pyspark.sql.dataframe import DataFrame 
from ml.slang_words import slang


emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad',
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed',
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink',
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

class Preprocessor(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    input_col = Param(Params._dummy(), "input_col", "input column name.", typeConverter=TypeConverters.toString)
    output_col = Param(Params._dummy(), "output_col", "output column name.", typeConverter=TypeConverters.toString)
    
    @keyword_only
    def __init__(self, input_col: str = "input", output_col: str = "output"):
        super(Preprocessor, self).__init__()
        self._setDefault(input_col=None, output_col=None)
        kwargs = self._input_kwargs
        self.set_params(**kwargs)

    @keyword_only
    def set_params(self, input_col: str = "input", output_col: str = "output"):
        kwargs = self._input_kwargs
        self._set(**kwargs)
        
    def get_input_col(self):
        return self.getOrDefault(self.input_col)
    
    def get_output_col(self):
        return self.getOrDefault(self.output_col)

    def remove_html_tags(self, text):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    
    def remove_url(self, text):
        pattern = re.compile(r'https?://\S+|www\.\S+')
        return pattern.sub(r'', text)
    
    def handle_emoji(self, text):
        for emoji in emojis.keys():
            text = text.replace(emoji, "EMOJI" + emojis[emoji])

        return text
    
    def chat_conversion(self, text):
        new_text = []
        for w in text.split():
            if w.upper() in slang:
                new_text.append(slang[w.upper()])
            else:
                new_text.append(w)
        return " ".join(new_text)
    
    def remove_punc(self, text):
        return text.translate(str.maketrans('', '', string.punctuation))
    
    def lowercase(self, text):
        return text.lower()
    
    def preprocess(self, text):
        text = self.remove_html_tags(text)
        text = self.remove_url(text)
        text = self.handle_emoji(text)
        text = self.chat_conversion(text)
        text = self.remove_punc(text)
        text = self.lowercase(text)

        return text 

    def _transform(self, df: DataFrame) -> DataFrame:
        input_col = self.get_input_col()
        output_col = self.get_output_col()

        transform_udf = F.udf(self.preprocess, StringType())

        return df.withColumn(output_col, transform_udf(input_col))

In [ ]:
def feature_extraction(df):
  preprocessor = Preprocessor(input_col="text", output_col="text")
  # Tokenize into words
  tokenizer = Tokenizer(inputCol="text", outputCol="tokenized")
  # Remove stopwords
  remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")
  # Compute term frequencies and hash into buckets
  hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="hashed",\
    numFeatures=1000)
  # Convert to TF-IDF
  idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="features")


  pipeline = Pipeline(stages=[preprocessor, tokenizer, remover, hashing_tf, idf])
  pipeline_model = pipeline.fit(df)

  tolist_udf = F.udf(
      lambda v: v.toArray().tolist(), 
      ArrayType(FloatType())
  )
  return pipeline_model.transform(df).\
    withColumn("features", tolist_udf("features")).cache()



In [ ]:
import pandas as pd 

class LabelerSimulator(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    input_col = Param(Params._dummy(), "input_col", "input column name.", typeConverter=TypeConverters.toString)
    output_col = Param(Params._dummy(), "output_col", "output column name.", typeConverter=TypeConverters.toString)
    
    @keyword_only
    def __init__(self, input_col: str = "input", output_col: str = "output"):
        super(Preprocessor, self).__init__()
        self._setDefault(input_col=None, output_col=None)
        kwargs = self._input_kwargs
        self.set_params(**kwargs)

    @keyword_only
    def set_params(self, input_col: str = "input", output_col: str = "output"):
        kwargs = self._input_kwargs
        self._set(**kwargs)
        
    def get_input_col(self):
        return self.getOrDefault(self.input_col)
    
    def get_output_col(self):
        return self.getOrDefault(self.output_col)


    def _transform(self, df: DataFrame) -> DataFrame:
        input_col = self.get_input_col()
        output_col = self.get_output_col()

        transform_udf = F.udf(self.preprocess, StringType())

        return df.withColumn(output_col, transform_udf(input_col))


# init dataset
df_tr = feature_extraction(df)
# X = df_tr['feature']


In [ ]:
df = spark.read.csv(
    'data/training_init.csv',
    header=True,
    inferSchema=True    
)


labels = spark.read.csv("data/label.csv",header=True,
    inferSchema=True    )

In [ ]:
from math import sqrt
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, f1_score
from hyperopt import fmin, hp, tpe
from hyperopt import SparkTrials, STATUS_OK

# Model
def build_test_train_split(df, val_size, test_size):
  # modAL needs numpy arrays; this converts M arrays of N-d arrays into MxN 2D array
  X = np.stack(df['features'].to_numpy())
  y = df['target'].to_numpy()
  
  (X_train, X2, y_train, y2) = train_test_split(
    X, y, 
    test_size = val_size + test_size, 
    stratify = y,
    random_state = 7
  )
  
  (X_val, X_test, y_val, y_test) = train_test_split(
    X2, y2, 
    test_size = test_size/(val_size + test_size),
    stratify = y2,
    random_state = 7
  )
  return (X_train, X_val, X_test, y_train, y_val, y_test)

# Core function to train a model given train set and params
def train_model(params, X_train, y_train):
  lr = LogisticRegression(solver='liblinear', max_iter=1000,\
                          penalty=params['penalty'], C=params['C'], random_state=7)
  return lr.fit(X_train, y_train)

# Use hyperopt to select a best model, given train/validation sets
def find_best_lr_model(X_train, X_val, y_train, y_val):
  # Wraps core modeling function to evaluate and return results for hyperopt
  def train_model_fmin(params):
    lr = train_model(params, X_train, y_train)
    loss = log_loss(y_val, lr.predict_proba(X_val))
    accuracy = accuracy_score(y_val, lr.predict(X_val))
    f1 = f1_score(y_val, lr.predict(X_val), pos_label=4)
    # supplement auto logging in mlflow with f1 and accuracy
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('accuracy', accuracy)
    return {
      'status': STATUS_OK, 
      'loss': loss, 
      'accuracy': accuracy, 
      'f1': f1
    }

  penalties = ['l1', 'l2']
  search_space = {
    'C': hp.loguniform('C', -6, 1),
    'penalty': hp.choice('penalty', penalties)
  }

  best_params = fmin(fn=train_model_fmin,
                     space=search_space,
                     algo=tpe.suggest,
                     max_evals=32,
                     trials=SparkTrials(parallelism=4),
                     rstate=np.random.default_rng(7))
  # Need to translate this back from 0/1 in output to be used again as input
  best_params['penalty'] = penalties[best_params['penalty']]
  # Train final model on train + validation sets
  final_model = train_model(best_params,\
                            np.concatenate([X_train, X_val]),\
                            np.concatenate([y_train, y_val]))
  return (best_params, final_model)

def log_and_eval_model(best_model, best_params, X_test, y_test):
  with mlflow.start_run():
    accuracy = accuracy_score(y_test, best_model.predict(X_test))
    f1 = f1_score(y_test, best_model.predict(X_test), pos_label=4)
    loss = log_loss(y_test, best_model.predict_proba(X_test))
    mlflow.log_params(best_params)
    mlflow.log_metrics({'accuracy': accuracy, 'log_loss': loss})
    mlflow.sklearn.log_model(best_model, "model")
    return (accuracy, f1, loss)

In [ ]:
from sklearn.metrics import classification_report, f1_score
print(f1_score(
    y_test, best_model.predict(X_test), pos_label=4
))
print(classification_report(
    y_test, best_model.predict(X_test)
))

In [ ]:

(X_train, X_val, X_test, y_train, y_val, y_test) = build_test_train_split(
    df_tr.toPandas(), 0.1, 0.1
)
(best_params, best_model) = find_best_lr_model(X_train, X_val, y_train, y_val)
(accuracy, f1, loss) = log_and_eval_model(best_model, best_params, X_test, y_test)


In [ ]:
log_and_eval_model(best_model, best_params, X_test, y_test)